In [ ]:
from os.path import join
import tensorflow as tf

class FCN:
    def __init__(self, batch_size, epochs, lr, regularization_const, saveDirectory, vgg_path):
        self.batch_size = batch_size
        self.epochs = epochs
        self.regularization_const = regularization_const
        self.saveDirectory = saveDirectory
        self.vgg_path = num_layers
        self.lr = lr
        
        self.global_step = None
        

        self.features_placeholder = None
        self.features_lengths_placeholder = None
        self.labels_placeholder = None
        self.prediction = None
        self.prediction_class = None
        self.prediction_softmax = None
        self.train_op = None

        self.loss = None
        self.accuracy = None

        self.saveDirectory = "checkpoint/", self.name)
    
    
    def load_model(self, sess):
        print("Will restore model from {path}".format(path=self.saveDirectory))
        training_saver = tf.train.Saver(max_to_keep=None)
        training_saver.restore(sess, self.saveDirectory)

    def save_model(self, sess, epoch):
        logging.info("Will save model at epoch {epoch} to {path}".format(epoch=epoch, path=self.saveDirectory))
        training_saver = tf.train.Saver(max_to_keep=None)
        training_saver.save(sess, self.saveDirectory, global_step=epoch)

    def train_model(self, sess, other_operations=list(), keep_prob=.5):
        operations_to_run = [self.train_op] + other_operations

        return sess.run(operations_to_run, feed_dict={
            self.keep_prob: keep_prob
        })
    
    def load_vgg(self, sess, vgg_path):
        """
        Load Pretrained VGG Model into TensorFlow.
        :param sess: TensorFlow Session
        :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
        :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
        """

        vgg_tag = 'vgg16'
        vgg_input_tensor_name = 'image_input:0'
        vgg_keep_prob_tensor_name = 'keep_prob:0'
        vgg_layer3_out_tensor_name = 'layer3_out:0'
        vgg_layer4_out_tensor_name = 'layer4_out:0'
        vgg_layer7_out_tensor_name = 'layer7_out:0'
        model = tf.saved_model.loader.load(
            sess,
            [vgg_tag],
            vgg_path
        )
        t1 = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
        t2 = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
        t3 = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
        t4 = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
        t5 = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)

        tf.summary.image('image', t1)
        tf.summary.histogram('vgg3', t3)
        tf.summary.histogram('vgg4', t4)
        tf.summary.histogram('vgg7', t5)

        return t1, t2,t3,t4,t5
        
class LSTM:
    def __init__(self,
                 batch_size,
                 vocabulary_size,
                 lstm_units,
                 max_input_series_length,
                 num_layers,
                 output_classes_count,
                 lr
                 ):
        self.batch_size = batch_size
        self.vocabulary_size = vocabulary_size
        self.lstm_units = lstm_units
        self.max_input_series_length = max_input_series_length
        self.num_layers = num_layers
        self.output_classes_count = output_classes_count
        self.lr = lr

        self.name = "LSTM_batch{batch}_vocab{vocab}_strpPunct{strip_punctuation}_lemm{lemmatize}_strpStopwords{strip_stopwords}" \
                    "_fwPadd{forward_padding}_maxLen{max_doc_length}_testSet{test_split_ratio}" \
                    "_nHidden{n_hidden_layers}_lstmUnits{lstm_units}_keepProb{keep_prob}_epochs{epochs}_lr{learning_rate}".format(
            batch=self.batch_size,
            vocab=self.vocabulary_size,
            max_doc_length=self.max_input_series_length,
            n_hidden_layers=self.num_layers,
            lstm_units=self.lstm_units,
            strip_punctuation=FLAGS.strip_punctuation,
            lemmatize=FLAGS.lemmatize,
            strip_stopwords=FLAGS.strip_stopwords,
            forward_padding=FLAGS.forward_padding,
            test_split_ratio=FLAGS.test_split_ratio,
            keep_prob=FLAGS.keep_prob,
            epochs=FLAGS.epochs,
            learning_rate=FLAGS.lr
        )


        self.global_step = None
        self.keep_prob = None


        self.features_placeholder = None
        self.features_lengths_placeholder = None
        self.labels_placeholder = None
        self.prediction = None
        self.prediction_class = None
        self.prediction_softmax = None
        self.train_op = None

        self.loss = None
        self.accuracy = None

        self.saveDirectory = join(getattr(FLAGS, 'job-dir'), "checkpoint", self.name)

    def build_model(self, features, features_length, labels):
        self.global_step = tf.Variable(0, name='global_step', trainable=False)

        embedding_size = self.lstm_units

        with tf.name_scope("input_data"):
            self.features_placeholder = features
            self.features_lengths_placeholder = features_length
            self.labels_placeholder = labels


            # self.features_placeholder = tf.placeholder(
            #     tf.int32,
            #     shape=[None, self.max_input_series_length],
            #     name="features_placeholder"
            # )
            # self.features_lengths_placeholder = tf.placeholder(
            #     tf.int32,
            #     shape=[None],
            #     name="features_lengths_placeholder"
            # )
            # self.labels_placeholder = tf.placeholder(tf.int32, shape=[None], name="labels_placeholder")

        with tf.name_scope("embedding"):
            word_embeddings = tf.get_variable("word_embeddings", [self.vocabulary_size, embedding_size])
            embedded_vecs = tf.nn.embedding_lookup(word_embeddings, self.features_placeholder)


        with tf.name_scope("hidden"):
            self.keep_prob = tf.placeholder_with_default(1.0, shape=())
            lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.lstm_units)
            lstm_cell = tf.contrib.rnn.DropoutWrapper(cell=lstm_cell, output_keep_prob=self.keep_prob)

            cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * self.num_layers)
            rnn_outputs, rnn_final_state = tf.nn.dynamic_rnn(
                cell,
                embedded_vecs,
                dtype=tf.float32,
                sequence_length=self.features_lengths_placeholder
            )

        with tf.name_scope("output_layer"):
            weight = tf.Variable(tf.truncated_normal([self.lstm_units, self.output_classes_count]), name="weight")
            bias = tf.Variable(tf.constant(0.1, shape=[self.output_classes_count]), name="bias")
            rnn_outputs = tf.transpose(rnn_outputs, [1, 0, 2])
            # last = tf.gather(rnn_outputs, int(rnn_outputs.get_shape()[0]) - 1)

            last = rnn_final_state[-1].h
            self.prediction = (tf.matmul(last, weight) + bias)
            self.prediction_class = tf.argmax(self.prediction, -1, output_type=tf.int32, name="prediction_class")

            self.prediction_softmax = tf.nn.softmax(self.prediction)

        with tf.name_scope("metrics"):
            correct_pred = tf.equal(self.prediction_class, self.labels_placeholder)
            self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                    logits=self.prediction,
                    labels=tf.one_hot(self.labels_placeholder, self.output_classes_count)
                )
            )

        tf.summary.scalar('Loss', self.loss)
        tf.summary.scalar('Accuracy', self.accuracy)

        self.train_op = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss, global_step=self.global_step)

    def load_model(self, sess):
        logging.info("Will restore model from {path}".format(path=self.saveDirectory))
        training_saver = tf.train.Saver(max_to_keep=None)
        training_saver.restore(sess, self.saveDirectory)

    def save_model(self, sess, epoch):
        logging.info("Will save model at epoch {epoch} to {path}".format(epoch=epoch, path=self.saveDirectory))
        training_saver = tf.train.Saver(max_to_keep=None)
        training_saver.save(sess, self.saveDirectory, global_step=epoch)

    def train_model(self, sess, other_operations=list(), keep_prob=.5):
        operations_to_run = [self.train_op] + other_operations

        return sess.run(operations_to_run, feed_dict={
            self.keep_prob: keep_prob
        })